In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
import pandas as pd

class Layer_Dense:

    #layer initialization 
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))

    #forward pass
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        #derivative wrt of weights is inputs. Dimension adjustment is needed
        self.dweights = np.dot(self.inputs.T, dvalues)
        #derivative of bias is column sums
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        #derivative wrt of inputs is weights
        self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0,inputs) #if the inputs is lower than 0, we make it 0, uf not, then we pass on

    def backward(self, dvalues):
        self.dinputs = dvalues.copy() #copy the gradient matrix
        self.dinputs[self.inputs<= 0] = 0 #if it is lower than 0 then we'll make it zero

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True)) #make the max = 1 and the min is -inf
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)# normalize

        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)

        for index, (single_output, single_dvalues)  in enumerate(zip(self.output, dvalues)):
            #single output is self.output[0]. it has the dimension 1xn, then the next line, we'll shift if to n,1
            single_output = single_output.reshape(-1,1) #reshape the output to become (,1). n row with 1 column
            
            #diagflat is to craete the matrix where diagonal is a value and the rest is 0. then based on the formula 
            #diagflat will have n,n matrix with diagonal is single_output and the rest is 0
            #then we have single.output where dim = n,1 and single.output.T dim = 1,n. this will result n,n
            jacobian_matrix  = np.diagflat(single_output) - np.dot(single_output, single_output.T)

            #jacobian matrix is n,n and single_values is n,1. Why Dvalues has n,n dimension and single is only the first row
            #this will result n,1 dimension
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output,y) #output in here is the preddiction
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        #clip data to avoid division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1 : 
            #softmax_outputs1 = np.array([[ 0.7 , 0.1 , 0.2 ],
            #[ 0.1 , 0.5 , 0.4 ],
            #[ 0.02 , 0.9 , 0.08 ]])
            #class_targets1 = np.array([ 0 , 1 , 1 ])
            #for example [0,1,1] just go to observation where it is y true. 
            #y_pred has a dimension nxn so for the first row, take index 0 y_pred_clipped[0,0]
            #y_pred_clipped[1,1]
            #y_pred_clipped[2,1]

            correct_cofidences = y_pred_clipped[ range(samples), y_true]   

        elif len(y_true.shape) == 2:
            #softmax_outputs2 = np.array([[ 0.7 , 0.1 , 0.2 ],
            #[ 0.1 , 0.5 , 0.4 ],
            #[ 0.02 , 0.9 , 0.08 ]])
            #class_targets2 = np.array([[ 1 , 0 , 0 ],
            #[ 0 , 1 , 0 ],
            #[ 0 , 1 , 0 ]])
            #since this is one hot encoding. only 1 value is 1 and the rest is zero. so when multiplying, only the given 1 will yield a result
            correct_confidences1 = np.sum(y_pred_clipped * y_true, axis = 1) 
        
        negative_log_likelihoods = -np.log(correct_cofidences)
        return negative_log_likelihoods
    
    def backward(self, dvalues, y_true):
        #number of samples
        samples = len(dvalues) #np.array([[ 1 , 2 , 3 ],[ 5 , 6 , 7 ],[ 9 , 10 , 11  ]])

        labels = len(dvalues[0]) #array([1,2,3])

        #if y_true is [0,1,1]
        #then np.eye will make it 
        #array([[1., 0., 0.],
        #       [0., 1., 0.],
        #       [0., 1., 0.]], dtype=float32)

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true] 
    
        self.dinputs = -y_true / dvalues #partial derivatives with respect tp inputs = matrix 3x3 - 3x3
        #the derivative of this loss fucntion with respect ot is input = ground truth vector / vector of predicted values

        self.dinputs = self.dinputs / samples
        #normalize to make the sum magnitude invariant to the number of samples. 

        
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs) #use softmax activation
        self.output = self.activation.output #the output is a probability
        return self.loss.calculate(self.output, y_true) #calculate loss between predicted (self.output) and y_true
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)

        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis = 1) #convert from one hot encoder to the discrete true labels
        
        self.dinputs = dvalues.copy()
        self.dinputs [range(samples) ,y_true] -= 1 #only at the given ytue, the value is minus by one. why?
        #becayse the partial derivative of loss wrt of softmax function inputs. 
        self.dinputs = self.dinputs/samples

class Optimizer_SGD:
    def __init__(self, learning_rate = 1):
        self.learning_rate = learning_rate
        self.name = "OptimizerSGD"

    def update_params(self, layer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases  


In [2]:
# Create dataset
X, y = spiral_data( samples = 100 , classes = 3 )
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense( 2 , 64 )
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense( 64 , 3 )
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
optimizer_class = [Optimizer_SGD( learning_rate = 1),
Optimizer_SGD( learning_rate = 0.85)
]


scores = {"method":[],"epoch":[],"loss":[],"lr":[],"acc":[]}

for optimizer in optimizer_class:
    for epoch in range ( 10001 ):
        # Perform a forward pass of our training data through this layer
        dense1.forward(X)
        # Perform a forward pass through activation function
        # takes the output of first dense layer here
        activation1.forward(dense1.output)
        # Perform a forward pass through second Dense layer
        # takes outputs of activation function of first layer as inputs
        dense2.forward(activation1.output)
        # Perform a forward pass through the activation/loss function
        # takes the output of second dense layer here and returns loss
        loss = loss_activation.forward(dense2.output, y)
        # Calculate accuracy from output of activation2 and targets
        # calculate values along first axis
        predictions = np.argmax(loss_activation.output, axis = 1 )
        if len (y.shape) == 2 :
            y = np.argmax(y, axis = 1 )
        accuracy = np.mean(predictions == y)
        if not epoch % 1000 :
            print (f'epoch: {epoch} , ' +
            f'acc: {accuracy :.3f} , ' +
            f'loss: {loss :.3f} ' )

            scores["method"].append(optimizer.name)
            scores["epoch"].append(epoch)
            scores["acc"].append(accuracy)
            scores["loss"].append(loss)
            scores["lr"].append(optimizer.learning_rate)    
                # Backward pass
        loss_activation.backward(loss_activation.output, y)
        dense2.backward(loss_activation.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        # Update weights and biases
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)

epoch: 0 , acc: 0.397 , loss: 1.099 
epoch: 1000 , acc: 0.457 , loss: 1.046 
epoch: 2000 , acc: 0.537 , loss: 0.897 
epoch: 3000 , acc: 0.637 , loss: 0.739 
epoch: 4000 , acc: 0.723 , loss: 0.604 
epoch: 5000 , acc: 0.767 , loss: 0.552 
epoch: 6000 , acc: 0.790 , loss: 0.506 
epoch: 7000 , acc: 0.803 , loss: 0.477 
epoch: 8000 , acc: 0.737 , loss: 0.763 
epoch: 9000 , acc: 0.820 , loss: 0.449 
epoch: 10000 , acc: 0.830 , loss: 0.428 
epoch: 0 , acc: 0.837 , loss: 0.439 
epoch: 1000 , acc: 0.840 , loss: 0.420 
epoch: 2000 , acc: 0.853 , loss: 0.411 
epoch: 3000 , acc: 0.850 , loss: 0.402 
epoch: 4000 , acc: 0.783 , loss: 0.817 
epoch: 5000 , acc: 0.863 , loss: 0.384 
epoch: 6000 , acc: 0.857 , loss: 0.387 
epoch: 7000 , acc: 0.857 , loss: 0.379 
epoch: 8000 , acc: 0.870 , loss: 0.374 
epoch: 9000 , acc: 0.697 , loss: 1.354 
epoch: 10000 , acc: 0.870 , loss: 0.359 


In [3]:
df_scores = pd.DataFrame(data = scores)
df_scores

,method,epoch,loss,lr,acc
0,OptimizerSGD,0,1.098605,1.00,0.396667
1,OptimizerSGD,1000,1.045727,1.00,0.456667
2,OptimizerSGD,2000,0.896628,1.00,0.536667
3,OptimizerSGD,3000,0.738755,1.00,0.636667
4,OptimizerSGD,4000,0.604224,1.00,0.723333
5,OptimizerSGD,5000,0.552417,1.00,0.766667
6,OptimizerSGD,6000,0.505868,1.00,0.790000
7,OptimizerSGD,7000,0.476831,1.00,0.803333
8,OptimizerSGD,8000,0.762996,1.00,0.736667
9,OptimizerSGD,9000,0.449063,1.00,0.820000
